In [1]:
# %pip install -r /home/huuquangdang/huu.quang.dang/thesis/deepfake/deepfake_backend/libs/model/CelebV2/requirements.txt

# # Step 1: Uninstall incompatible NumPy version
# %pip uninstall numpy -y
# # Step 2: Install compatible NumPy version (< 2.0.0)
# %pip install "numpy>=1.21.0,<2.0.0"
# # Step 3: Verify NumPy version
# import numpy as np
# print(f"✅ NumPy version: {np.__version__}")
# print(f"Expected: < 2.0.0 (you should see 1.x.x)")

In [2]:
# Built-in
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, TimeDistributed, GlobalAveragePooling2D, LSTM, Dense, Dropout
)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Scikit-learn
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

2025-10-27 21:43:34.413447: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 21:43:34.466144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 21:43:34.791770: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-27 21:43:34.791834: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-27 21:43:34.793456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
# Define the path to the dataset
base_path = '/home/huuquangdang/huu.quang.dang/thesis/Dataset/facep/faceplus_processed_research_final_2_crop'
categories = ['fake', 'real']

# Initialize a list to hold data
data = []

# Process each category
for category in categories:
    category_path = os.path.join(base_path, category)
    for filename in os.listdir(category_path):
        if filename.endswith('.jpg'):
            try:
                id_part, frame_part = filename.split('_frame_')
                id_ = id_part.split('_')[0]
                frame = frame_part.split('.')[0]
                data.append({
                    'filename': filename,
                    'path': os.path.join(category_path, filename),
                    'id': int(id_),
                    'frame': int(frame),
                    'label': category
                })
            except ValueError:
                continue

# Convert the data to a DataFrame
df = pd.DataFrame(data)

In [4]:
# Đảm bảo đã có df_cropped.csv chứa đường dẫn ảnh đã crop
df['label_id'] = df['label'].map({'fake': 0, 'real': 1})
df


,filename,path,id,frame,label,label_id
0,855_801_frame_0018.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,855,18,fake,0
1,686_696_frame_0024.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,686,24,fake,0
2,664_668_frame_0017.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,664,17,fake,0
3,486_680_frame_0010.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,486,10,fake,0
4,464_463_frame_0005.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,464,5,fake,0
...,...,...,...,...,...,...
59803,140_frame_0025.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,140,25,real,1
59804,937_frame_0007.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,937,7,real,1
59805,092_frame_0001.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,92,1,real,1
59806,736_frame_0018.jpg,/home/huuquangdang/huu.quang.dang/thesis/Datas...,736,18,real,1


In [5]:
df['video_key'] = df['id'].astype(str) + "_" + df['label']

from collections import defaultdict

video_dict = defaultdict(list)
labels = {}

for _, row in df.iterrows():
    key = row['video_key']
    video_dict[key].append(row['path'])
    labels[key] = row['label_id']


In [6]:
# !pip install opencv-python

In [7]:
# Load pre-extracted CSV features for feature fusion
print("📂 Loading pre-extracted OpenFace features from CSV files...")
csv_fake_path = '/home/huuquangdang/huu.quang.dang/thesis/deepfake/deepfake_backend/libs/tools/fake/fpp_fake_v1.csv'
csv_real_path = '/home/huuquangdang/huu.quang.dang/thesis/deepfake/deepfake_backend/libs/tools/real/fpp_real_v1.csv'

df_fake_features = pd.read_csv(csv_fake_path, header=None)
df_real_features = pd.read_csv(csv_real_path, header=None)

# Combine both dataframes
df_all_features = pd.concat([df_fake_features, df_real_features], ignore_index=True)
df_all_features.columns = ['filename'] + [f'feat_{i}' for i in range(df_all_features.shape[1] - 1)]

# Create a dictionary for fast lookup: filename -> features
openface_features = {}
for _, row in df_all_features.iterrows():
    filename = row['filename']
    features = row.iloc[1:].values.astype('float32')
    openface_features[filename] = features

csv_feature_dim = len(features)
print(f"✅ Loaded {len(openface_features)} feature vectors")
print(f"✅ Each vector has {csv_feature_dim} dimensions")
print(f"✅ Feature dictionary ready for 1-to-1 frame mapping")

📂 Loading pre-extracted OpenFace features from CSV files...
✅ Loaded 59808 feature vectors
✅ Each vector has 674 dimensions
✅ Feature dictionary ready for 1-to-1 frame mapping


In [ ]:
# 🔍 Debug: Check filename matching between images and CSV features
print("🔍 Debugging filename matching:")
print(f"\n📁 Sample image filenames (from video_dict):")
sample_paths = list(video_dict.values())[0][:3]
for path in sample_paths:
    print(f"   - {os.path.basename(path)}")

print(f"\n📄 Sample CSV feature keys (from openface_features):")
sample_csv_keys = list(openface_features.keys())[:5]
for key in sample_csv_keys:
    print(f"   - {key}")

# Check for matches
print(f"\n🔎 Checking for matches:")
matches = 0
for path in sample_paths:
    filename = os.path.basename(path)
    if filename in openface_features:
        print(f"   ✅ MATCH: {filename}")
        matches += 1
    else:
        print(f"   ❌ NO MATCH: {filename}")
        # Try to find similar names
        similar = [k for k in list(openface_features.keys())[:10] if filename[:10] in k or k[:10] in filename]
        if similar:
            print(f"      Similar: {similar[:3]}")

print(f"\n📊 Total matches: {matches}/{len(sample_paths)}")

🔍 Debugging filename matching:

📁 Sample image filenames (from video_dict):
   - 855_801_frame_0018.jpg
   - 855_801_frame_0011.jpg
   - 855_801_frame_0016.jpg

📄 Sample CSV feature keys (from openface_features):
   - 000_003_frame_0000.jpg
   - 000_003_frame_0001.jpg
   - 000_003_frame_0002.jpg
   - 000_003_frame_0003.jpg
   - 000_003_frame_0004.jpg

🔎 Checking for matches:
   ✅ MATCH: 855_801_frame_0018.jpg
   ✅ MATCH: 855_801_frame_0011.jpg
   ✅ MATCH: 855_801_frame_0016.jpg

📊 Total matches: 3/3


: 

# 🔬 Model: BiGRU + Multi-Head Self-Attention

This cell implements **BiGRU with Multi-Head Self-Attention mechanism** for improved sequence modeling, while maintaining:
- ✅ Same function names (`VideoSequence`, `build_model`, `hmm_postprocess`)
- ✅ Same 1-to-1 feature fusion (MobileNet + CSV)
- ✅ Same normalization strategy
- ✅ Enhanced with Bidirectional GRU and Multi-Head Self-Attention layer

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (
    Input, TimeDistributed, GRU, Dropout, Dense, GlobalAveragePooling2D, 
    Concatenate, BatchNormalization, Bidirectional, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
    )
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from hmmlearn.hmm import GaussianHMM
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler

# Configuration - Enhanced
video_keys = list(video_dict.keys())
video_labels = [labels[k] for k in video_keys]

img_size = (224, 224)
batch_size = 16  # Reduced for better generalization
epochs = 100  # Increased for better convergence
n_splits = 5
sequence_len = 30  # Increased to capture more temporal information
results = []
all_histories = []

# Normalize CSV features with robust scaling
print("🔧 Computing CSV feature statistics for normalization...")
all_csv_features = []
for key in video_keys:
    frames = video_dict[key][:sequence_len]
    for path in frames:
        filename = os.path.basename(path)
        if filename in openface_features:
            all_csv_features.append(openface_features[filename])

# Debug: Check if we have any features
print(f"📊 Found {len(all_csv_features)} CSV features from {len(video_keys)} videos")
if len(all_csv_features) == 0:
    print("⚠️ WARNING: No matching CSV features found!")
    print(f"   Sample video filename: {os.path.basename(list(video_dict.values())[0][0])}")
    print(f"   Sample CSV key: {list(openface_features.keys())[0]}")
    raise ValueError("No CSV features found - check filename matching!")

csv_scaler = StandardScaler()
csv_scaler.fit(np.array(all_csv_features))  # Convert to numpy array
print(f"✅ CSV feature normalization fitted on {len(all_csv_features)} samples")

# Enhanced Data generator with stronger augmentation
class VideoSequence(tf.keras.utils.Sequence):
    def __init__(self, video_keys, video_dict, labels, batch_size, img_size, sequence_len=15, augment=False):
        self.video_keys = video_keys
        self.video_dict = video_dict
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.sequence_len = sequence_len
        self.augment = augment
        # Enhanced augmentation for better regularization
        self.datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20 if augment else 0,
            width_shift_range=0.15 if augment else 0,
            height_shift_range=0.15 if augment else 0,
            zoom_range=0.15 if augment else 0,
            horizontal_flip=augment,
            brightness_range=[0.8, 1.2] if augment else None,
            fill_mode='nearest'
        )

    def __len__(self):
        return int(np.ceil(len(self.video_keys) / self.batch_size))

    def __getitem__(self, idx):
        batch_keys = self.video_keys[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_X_img, batch_X_csv, batch_y = [], [], []

        for key in batch_keys:
            frames = self.video_dict[key][:self.sequence_len]
            imgs = []
            csv_feats = []
            
            for path in frames:
                # Load and process image
                img = cv2.imread(path)
                img = cv2.resize(img, self.img_size)
                img = self.datagen.random_transform(img) if self.augment else img
                img = img.astype('float32') / 255.0
                imgs.append(img)
                
                # Load CSV features and normalize (1-to-1 mapping)
                filename = os.path.basename(path)
                if filename in openface_features:
                    csv_feat = openface_features[filename]
                    csv_feat = csv_scaler.transform(csv_feat.reshape(1, -1))[0]
                else:
                    csv_feat = np.zeros(csv_feature_dim, dtype='float32')
                csv_feats.append(csv_feat)
            
            # Pad sequences if needed
            while len(imgs) < self.sequence_len:
                imgs.append(np.zeros((*self.img_size, 3), dtype='float32'))
                csv_feats.append(np.zeros(csv_feature_dim, dtype='float32'))
            
            batch_X_img.append(imgs)
            batch_X_csv.append(csv_feats)
            batch_y.append(self.labels[key])

        return [np.array(batch_X_img), np.array(batch_X_csv)], np.array(batch_y)

# Enhanced BiGRU + Multi-Head Self-Attention Model
def build_model(sequence_len, img_size, csv_dim=674):
    # MobileNet branch with more trainable layers
    base_cnn = MobileNetV2(input_shape=(*img_size, 3), include_top=False, weights='imagenet')
    base_cnn.trainable = True
    # Fine-tune more layers for better feature extraction
    for layer in base_cnn.layers[:-30]:
        layer.trainable = False
    
    cnn_out = GlobalAveragePooling2D()(base_cnn.output)
    cnn_model = Model(inputs=base_cnn.input, outputs=cnn_out)
    
    # Image sequence input
    input_seq_img = Input(shape=(sequence_len, *img_size, 3), name='image_input')
    x_img = TimeDistributed(cnn_model)(input_seq_img)
    
    # CSV features input
    input_seq_csv = Input(shape=(sequence_len, csv_dim), name='csv_input')
    
    # BatchNormalization before fusion
    x_img = BatchNormalization(name='bn_mobilenet')(x_img)
    x_csv = BatchNormalization(name='bn_csv')(input_seq_csv)
    
    # Concatenate features (1-to-1 fusion)
    x_combined = Concatenate(axis=-1, name='feature_fusion')([x_img, x_csv])
    
    # Enhanced BiGRU layers with more capacity
    x = Bidirectional(GRU(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, name='bigru_1'))(x_combined)
    x = LayerNormalization(name='ln_bigru_1')(x)
    x = Dropout(0.3, name='dropout_1')(x)
    
    x = Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, name='bigru_2'))(x)
    x = LayerNormalization(name='ln_bigru_2')(x)
    x = Dropout(0.3, name='dropout_2')(x)
    
    # Enhanced Multi-Head Self-Attention with more heads
    x_norm = LayerNormalization(name='ln_before_attention')(x)
    attn_output = MultiHeadAttention(
        num_heads=8,  # Increased from 4 to 8 for better pattern capture
        key_dim=64,   # Increased from 32 to 64
        dropout=0.1,
        name='multi_head_attention'
    )(x_norm, x_norm)
    
    # Residual connection + Layer normalization
    x = tf.keras.layers.Add(name='residual_connection')([x, attn_output])
    x = LayerNormalization(name='ln_after_attention')(x)
    x = Dropout(0.2, name='dropout_attention_1')(x)
    
    # Additional attention layer for deeper understanding
    x_norm2 = LayerNormalization(name='ln_before_attention_2')(x)
    attn_output2 = MultiHeadAttention(
        num_heads=4,
        key_dim=32,
        dropout=0.1,
        name='multi_head_attention_2'
    )(x_norm2, x_norm2)
    
    x = tf.keras.layers.Add(name='residual_connection_2')([x, attn_output2])
    x = LayerNormalization(name='ln_after_attention_2')(x)
    
    # Global average pooling over time dimension
    x = GlobalAveragePooling1D(name='global_avg_pool')(x)
    x = Dropout(0.3, name='dropout_attention_2')(x)
    
    # Enhanced classification layers with more capacity
    x = Dense(128, activation='relu', name='dense_1')(x)
    x = BatchNormalization(name='bn_dense_1')(x)
    x = Dropout(0.4, name='dropout_3')(x)
    
    x = Dense(64, activation='relu', name='dense_2')(x)
    x = BatchNormalization(name='bn_dense_2')(x)
    x = Dropout(0.3, name='dropout_4')(x)
    
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[input_seq_img, input_seq_csv], outputs=output, name='BiGRU_MultiHeadAttn')
    return model

# HMM post-processing (SAME NAME, unchanged)
def hmm_postprocess(pred_probs, y_true, n_states=2):
    pred_probs = pred_probs.reshape(-1, 1)
    hmm = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100)
    hmm.fit(pred_probs)
    hidden_states = hmm.predict(pred_probs)

    mapping = {}
    for state in np.unique(hidden_states):
        indices = [i for i in range(len(hidden_states)) if hidden_states[i] == state]
        state_labels = [y_true[i] for i in indices]
        if len(state_labels) > 0:
            mapped_label = mode(state_labels, keepdims=True).mode[0]
        else:
            mapped_label = 0
        mapping[state] = mapped_label

    hmm_labels = np.array([mapping[s] for s in hidden_states])
    return hmm_labels

# Training K-Fold
print("\n🚀 Starting Enhanced BiGRU + Multi-Head Self-Attention Training:")
print("   ✅ Bidirectional GRU (256 + 128 units) with recurrent dropout")
print("   ✅ Dual Multi-Head Self-Attention (8 heads + 4 heads)")
print("   ✅ Layer Normalization after each major block")
print("   ✅ Enhanced data augmentation")
print("   ✅ Sequence length: 15 frames")
print("   ✅ MobileNet (last 30 layers trainable)")
print("   ✅ Stronger regularization (dropout 0.3-0.4)")
print("   ✅ HMM post-processing\n")

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (trainval_idx, test_idx) in enumerate(skf.split(video_keys, video_labels), 1):
    print(f"\n{'='*60}")
    print(f"📊 Fold {fold}/{n_splits} - Enhanced BiGRU_MultiHeadAttn")
    print(f"{'='*60}")

    trainval_keys = [video_keys[i] for i in trainval_idx]
    test_keys = [video_keys[i] for i in test_idx]

    y_trainval = [labels[k] for k in trainval_keys]
    train_keys, val_keys = train_test_split(trainval_keys, test_size=0.15, stratify=y_trainval, random_state=fold)

    train_gen = VideoSequence(train_keys, video_dict, labels, batch_size, img_size, sequence_len, augment=True)
    val_gen = VideoSequence(val_keys, video_dict, labels, batch_size, img_size, sequence_len, augment=False)
    test_gen = VideoSequence(test_keys, video_dict, labels, batch_size, img_size, sequence_len, augment=False)

    model = build_model(sequence_len, img_size, csv_feature_dim)
    
    # Enhanced optimizer with learning rate warmup
    initial_lr = 1e-4
    model.compile(
        optimizer=Adamax(learning_rate=initial_lr),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"\n📈 Enhanced Model Architecture:")
    print(f"   - Model: BiGRU (256+128) + Dual Attention (8+4 heads)")
    print(f"   - Total params: {model.count_params():,}")
    trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
    print(f"   - Trainable params: {trainable_params:,}")
    print(f"   - Sequence length: {sequence_len}")
    print(f"   - Batch size: {batch_size}")
    print(f"   - Initial learning rate: {initial_lr}")

    model_path = f"best_model_fold{fold}_bigru_mhattn_enhanced.h5"
    checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)

    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs,
                        callbacks=[checkpoint, earlystop, reduce_lr], verbose=1)
    all_histories.append(history.history)

    model.load_weights(model_path)

    y_true = [labels[k] for k in test_keys]
    y_pred_prob = model.predict(test_gen).ravel()
    y_hmm_pred = hmm_postprocess(y_pred_prob, y_true)

    results.append({
        'fold': fold,
        'accuracy': accuracy_score(y_true, y_hmm_pred),
        'precision': precision_score(y_true, y_hmm_pred),
        'recall': recall_score(y_true, y_hmm_pred),
        'f1': f1_score(y_true, y_hmm_pred),
        'auc': roc_auc_score(y_true, y_pred_prob)
    })
    
    print(f"\n✅ Fold {fold} Results (Enhanced BiGRU_MultiHeadAttn):")
    print(f"   Accuracy:  {results[-1]['accuracy']:.4f}")
    print(f"   Precision: {results[-1]['precision']:.4f}")
    print(f"   Recall:    {results[-1]['recall']:.4f}")
    print(f"   F1 Score:  {results[-1]['f1']:.4f}")
    print(f"   AUC:       {results[-1]['auc']:.4f}")

print(f"\n{'='*60}")
print("📊 FINAL RESULTS - Enhanced BiGRU_MultiHeadAttn Model")
print(f"{'='*60}")
for r in results:
    print(f"Fold {r['fold']}: Acc={r['accuracy']:.4f}, Prec={r['precision']:.4f}, Rec={r['recall']:.4f}, F1={r['f1']:.4f}, AUC={r['auc']:.4f}")

print(f"\n📊 Average Metrics:")
results_df = pd.DataFrame(results)
print(results_df.mean(numeric_only=True))

# Calculate CV metrics
accuracy_mean = results_df['accuracy'].mean()
accuracy_std = results_df['accuracy'].std()
accuracy_range = results_df['accuracy'].max() - results_df['accuracy'].min()
accuracy_cv_percent = (accuracy_std / accuracy_mean) * 100

print(f"\n📈 Cross-Validation Stability:")
print(f"   Mean Accuracy: {accuracy_mean:.4f}")
print(f"   Std Deviation: {accuracy_std:.4f}")
print(f"   Range: {accuracy_range:.4f}")
print(f"   CV%: {accuracy_cv_percent:.2f}%")

🔧 Computing CSV feature statistics for normalization...
📊 Found 59808 CSV features from 2000 videos
✅ CSV feature normalization fitted on 59808 samples

🚀 Starting Enhanced BiGRU + Multi-Head Self-Attention Training:
   ✅ Bidirectional GRU (256 + 128 units) with recurrent dropout
   ✅ Dual Multi-Head Self-Attention (8 heads + 4 heads)
   ✅ Layer Normalization after each major block
   ✅ Enhanced data augmentation
   ✅ Sequence length: 15 frames
   ✅ MobileNet (last 30 layers trainable)
   ✅ Stronger regularization (dropout 0.3-0.4)
   ✅ HMM post-processing


📊 Fold 1/5 - Enhanced BiGRU_MultiHeadAttn

📈 Enhanced Model Architecture:
   - Model: BiGRU (256+128) + Dual Attention (8+4 heads)
   - Total params: 6,859,849
   - Trainable params: 6,123,973
   - Sequence length: 30
   - Batch size: 16
   - Initial learning rate: 0.0001
Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 0.8410 - accuracy: 0.5809
Epoch 1: val_accuracy improved from -inf to 0.60833, saving model t

/home/huuquangdang/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 524s 6s/step - loss: 0.8410 - accuracy: 0.5809 - val_loss: 0.7738 - val_accuracy: 0.6083 - lr: 1.0000e-04
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.6140 - accuracy: 0.6904
Epoch 2: val_accuracy improved from 0.60833 to 0.62917, saving model to best_model_fold1_bigru_mhattn_enhanced.h5
85/85 [==============================] - 503s 6s/step - loss: 0.6140 - accuracy: 0.6904 - val_loss: 0.8694 - val_accuracy: 0.6292 - lr: 1.0000e-04
Epoch 3/100


/home/huuquangdang/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - ETA: 0s - loss: 0.5261 - accuracy: 0.7581
Epoch 3: val_accuracy improved from 0.62917 to 0.68750, saving model to best_model_fold1_bigru_mhattn_enhanced.h5
85/85 [==============================] - 500s 6s/step - loss: 0.5261 - accuracy: 0.7581 - val_loss: 1.0481 - val_accuracy: 0.6875 - lr: 1.0000e-04
Epoch 4/100


...

Epoch 12/100
85/85 [==============================] - ETA: 0s - loss: 0.2278 - accuracy: 0.9059
Epoch 12: val_accuracy improved from 0.84583 to 0.85000, saving model to best_model_fold1_bigru_mhattn_enhanced.h5
85/85 [==============================] - 503s 6s/step - loss: 0.2278 - accuracy: 0.9059 - val_loss: 0.5389 - val_accuracy: 0.8500 - lr: 5.0000e-05
Epoch 13/100


/home/huuquangdang/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - ETA: 0s - loss: 0.2333 - accuracy: 0.9096
Epoch 13: val_accuracy did not improve from 0.85000
85/85 [==============================] - 501s 6s/step - loss: 0.2333 - accuracy: 0.9096 - val_loss: 0.5905 - val_accuracy: 0.8417 - lr: 5.0000e-05
Epoch 14/100
85/85 [==============================] - ETA: 0s - loss: 0.2219 - accuracy: 0.9051
Epoch 14: val_accuracy improved from 0.85000 to 0.86667, saving model to best_model_fold1_bigru_mhattn_enhanced.h5
85/85 [==============================] - 500s 6s/step - loss: 0.2219 - accuracy: 0.9051 - val_loss: 0.4925 - val_accuracy: 0.8667 - lr: 5.0000e-05


/home/huuquangdang/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 15/100
85/85 [==============================] - ETA: 0s - loss: 0.2039 - accuracy: 0.9279

In [ ]:
results_df = pd.DataFrame(results)
print("📊 Kết quả trung bình:")
print(results_df.mean(numeric_only=True))
results_df


In [ ]:
for i, hist in enumerate(all_histories, 1):
    plt.figure()
    plt.plot(hist['accuracy'], label='Train Acc')
    plt.plot(hist['val_accuracy'], label='Val Acc')
    plt.title(f'Fold {i} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure()
    plt.plot(hist['loss'], label='Train Loss')
    plt.plot(hist['val_loss'], label='Val Loss')
    plt.title(f'Fold {i} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# Save mô hình fold cuối cùng
model.save("mobilenetv2_hmm_faceplus_final.h5")


In [ ]:
results_df = pd.DataFrame(results)
print("📊 Kết quả trung bình:")
print(results_df.mean(numeric_only=True))
results_df


In [ ]:
import pandas as pd

# Giả sử results đã có và bạn đã tạo results_df
results_df = pd.DataFrame(results)

# Tính các chỉ số
accuracy_mean = results_df['accuracy'].mean()
accuracy_std = results_df['accuracy'].std()  # dùng sample std (chia cho n-1)
accuracy_range = results_df['accuracy'].max() - results_df['accuracy'].min()
accuracy_cv_percent = (accuracy_std / accuracy_mean) * 100

# In kết quả
print("📊 Kết quả trung bình:")
print(results_df.mean(numeric_only=True))

print(f"\n✅ CV Accuracy (Mean Accuracy): {accuracy_mean:.4f}")
print(f"📈 Range Accuracy: {accuracy_range:.4f}")
print(f"📉 Accuracy CV% (std/mean): {accuracy_cv_percent:.2f}%")

# Hiển thị bảng kết quả nếu cần
results_df
